In [70]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ''

from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"] = ''
# 配置模型地址
os.environ["OPENAI_API_BASE"] = "https://api.gpts.vin/v1"
# 配置模型版本
model = ChatOpenAI(model="gpt-3.5-turbo")

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a good assistant."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [71]:
from langchain_core.messages import SystemMessage,trim_messages,HumanMessage,AIMessage
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content="I am Bob!"),
    AIMessage(content="hi!"),
    HumanMessage(content="i like vanilla ice cream"),
    AIMessage(content="NICE!"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="you are welcome"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content='you are a good assistant'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='you are welcome'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [72]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)
response = chain.invoke({
    "messages": messages + [HumanMessage(content="what,s my name?")],
    "language": "english",
})
response.content

"I'm sorry, but I don't have access to that information. If you've mentioned your name before, I'm unable to remember it in our conversation."

In [73]:
response = chain.invoke({
    "messages": messages + [HumanMessage(content="what math problem did i ask?")],
    "language": "english",
})
response.content

'You asked "what\'s 2 + 2?"'

In [74]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory,InMemoryChatMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [75]:

response = with_message_history.invoke({
    "messages": messages + [HumanMessage(content="whats my name?")],
    "language": "English",
},config=config)
response.content

response = with_message_history.invoke({
    "messages": messages + [HumanMessage(content="what math problem did i ask?")],
    "language": "English",
},config=config)

response.content


'You asked "what\'s 2 + 2?"'